In [19]:
#First, we'll verify that you are running with an NVIDIA GPU.
!nvidia-smi 

Tue Dec 26 13:48:13 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P0              12W /  60W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

TESTING GYM ENVIRONMENT BEFORE APPLYING REINFORCEMENT LEARNING FOR CART POLE GAME FROM "https://gymnasium.farama.org/environments/classic_control/cart_pole/"

In [1]:
import gym 
import random

In [2]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n
print("States:{} Action:{}".format(states,actions))

States:4 Action:2


In [3]:
env.reset() #To reset env

array([ 0.00973164, -0.03580954, -0.02696606,  0.04522312])

In [4]:
import gym # USE 2020 MAY VERSION FOR VISUALIZATION 
import random
import time  # Add this import

env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

episodes = 10

for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward

        # Add a delay to slow down rendering
        time.sleep(0.01)  # Adjust the sleep duration as needed

    print('Episode:{} Score:{}'.format(episode, score))

env.close()  

Episode:1 Score:19.0
Episode:2 Score:19.0
Episode:3 Score:15.0
Episode:4 Score:13.0
Episode:5 Score:19.0
Episode:6 Score:22.0
Episode:7 Score:12.0
Episode:8 Score:17.0
Episode:9 Score:21.0
Episode:10 Score:27.0


In [12]:
import gym #Alternate version for new GYM VERSION

env = gym.make('CartPole-v1')
env.reset()

for i in range(20):
    # env.action_space.sample() produces either 0 (left) or 1 (right).
    step_result = env.step(env.action_space.sample())
    print("step", i, step_result)

env.close()

step 0 (array([ 0.01394968,  0.24340105, -0.0090812 , -0.32946035]), 1.0, False, {})
step 1 (array([ 0.0188177 ,  0.04840954, -0.0156704 , -0.03965498]), 1.0, False, {})
step 2 (array([ 0.01978589, -0.14648423, -0.0164635 ,  0.24804287]), 1.0, False, {})
step 3 (array([ 0.01685621, -0.34136724, -0.01150264,  0.53548772]), 1.0, False, {})
step 4 (array([ 0.01002886, -0.14608544, -0.00079289,  0.2392027 ]), 1.0, False, {})
step 5 (array([ 0.00710716, -0.34119606,  0.00399116,  0.53163542]), 1.0, False, {})
step 6 (array([ 2.83234445e-04, -5.36373920e-01,  1.46238725e-02,  8.25573283e-01]), 1.0, False, {})
step 7 (array([-0.01044424, -0.73169279,  0.03113534,  1.12281953]), 1.0, False, {})
step 8 (array([-0.0250781 , -0.53699259,  0.05359173,  0.84006309]), 1.0, False, {})
step 9 (array([-0.03581795, -0.73280367,  0.07039299,  1.14910644]), 1.0, False, {})
step 10 (array([-0.05047402, -0.53866759,  0.09337512,  0.87930196]), 1.0, False, {})
step 11 (array([-0.06124738, -0.73492573,  0.110

c:\Users\Pritish\anaconda3\envs\ML5\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [5]:
def build_model(states, action):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation="linear"))
    return model

In [14]:
model = build_model(states, action)

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770 (3.01 KB)
Trainable params: 770 (3.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 80000, window_length=1)
    dqn = DQNAgent(model=model,memory=memory,policy=policy,
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [31]:
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=80000, visualize=False, verbose=1)

Training for 80000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 196.080 [49.000, 200.000] - loss: 16.316 - mae: 43.941 - mean_q: 87.729

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 15.674 - mae: 42.053 - mean_q: 83.761

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: 1.0000
57 episodes - episode_reward: 178.035 [15.000, 200.000] - loss: 15.594 - mae: 40.377 - mean_q: 80.354

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 46s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 15.025 - mae: 39.639 - mean_q: 79.134

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 51s 5ms/step - reward: 1.0000
5

In [33]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
200.0


In [34]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [35]:
del model
del dqn
del env

In [6]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=1e-3), metrics=['mae'])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [7]:
dqn.load_weights('dqn_weights.h5f')

In [8]:
_ = dqn.test(env, nb_episodes=5, visualize=False)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200


c:\Users\Pritish\anaconda3\envs\ML5\lib\site-packages\keras\engine\training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
